In [1]:
import random

import numpy as np
import pandas as pd
from scipy import sparse

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

In [2]:
import math
# ground_truth: list of items ordered by time
def nDCG_Time(ground_truth, _recList):
    rec_num = len(_recList) # topK
    # ground_truth is already sorted by time
    idealOrder = ground_truth
    idealDCG = 0.0
    for j in range(min(rec_num, len(idealOrder))):
        idealDCG += ((math.pow(2.0, len(idealOrder) - j) - 1) / math.log(2.0 + j))

    recDCG = 0.0
    for j in range(rec_num):
        item = _recList[j]
        if item in ground_truth:
            rank = len(ground_truth) - ground_truth.index(item) # why ground truth?
            recDCG += ((math.pow(2.0, rank) - 1) / math.log(1.0 + j + 1))

    return (recDCG / idealDCG)


def Recall(_test_set, _recList):
    hit = len(set(_recList).intersection(set(_test_set)))
    return hit / float(len(_test_set))


def Precision(_test_set, _recList):
    hit = len(set(_recList).intersection(set(_test_set)))
    return hit / float(len(_recList))

In [3]:
ratings = pd.read_csv('ml-1m/ratings.dat', header=None, sep='::', 
                      encoding='latin-1', engine='python', 
                      names=['user_id', 'movie_id', 'rating', 'timestamp'])
users = pd.read_csv('ml-1m/users.dat', header=None, sep='::',
                    encoding='latin-1', engine='python',
                    names=['user_id', 'gender', 'age', 'occupation', 'zipcode'])
movies = pd.read_csv('ml-1m/movies.dat', header=None, sep='::',
                     encoding='latin-1', engine='python',
                     names=['movie_id', 'title', 'genres'])

In [4]:
ratings['user_id'] = ratings['user_id'] - 1
ratings['movie_id'] = ratings['movie_id'] - 1
users['user_id'] = users['user_id'] - 1
movies['movie_id'] = movies['movie_id'] - 1

In [5]:
user_n = ratings['user_id'].nunique()
item_n = movies['movie_id'].nunique()

In [6]:
item_id_to_iid = {movies['movie_id'][i]:i for i in movies.index}
item_iid_to_id = {i:movies['movie_id'][i] for i in movies.index}

In [7]:
tf = TfidfVectorizer(analyzer=lambda x: (g for g in x.split('|')))

In [8]:
analyzer = tf.build_analyzer()

In [9]:
X_tfidf = tf.fit_transform(movies['genres'])